# 08 — Classes & Instances

Goal: Understand how to define your own types using `class`, how instances store data, and how methods work.

This is the bridge from:
- “I know what a function is”
to:
- “I can define a `Layer`, `Optimizer`, or `Dataset` object that behaves like things in NNFS / PyTorch / Keras.”


## 1. What Is a Class?

A **class** is a blueprint for creating objects (instances).

- The **class** defines what data and behaviour objects of that type have.
- An **instance** is a specific object created from the class.

Example idea:

- Class: `Dog` → what it means to be a dog.
- Instance: `my_dog` → one particular dog.

In [1]:
class Dog:
    pass

# Create instances
d1 = Dog()
d2 = Dog()

print(d1)
print(d2)
print("Are they the same object?", d1 is d2)


Are they the same object? False


## 2. `__init__` and `self`

`__init__` is the **initializer**: it runs when you create an instance.

```python
class Dog:
    def __init__(self, name):
        self.name = name
```
- `self` is the instance being created.

- `self.name` becomes an **instance attribute**.

In [2]:
class Dog:
    def __init__(self, name):
        self.name = name

dog1 = Dog("Rex")
dog2 = Dog("Luna")

print(dog1.name)
print(dog2.name)

Rex
Luna


## 3. Instance Attributes

Attributes set on `self` inside `__init__` are **per-instance**:

```python
self.name = name
self.age = age
```
Each instance gets its own copy:

- `dog1.name` and `dog2.name` can be different.

- Changing one does not change the other.

In [3]:
class Counter:
    def __init__(self):
        self.value = 0

    def increment(self):
        self.value += 1

c1 = Counter()
c2 = Counter()

c1.increment()
c1.increment()
c2.increment()

print("c1 value:", c1.value)  # 2
print("c2 value:", c2.value)  # 1

c1 value: 2
c2 value: 1


## 4. Methods

A **method** is just a function defined inside a class.

```python
class Greeter:
    def __init__(self, name):
        self.name = name

    def greet(self):
        print(f"Hi, I'm {self.name}")
```
You call it on an instance:
```python
g = Greeter("Joe")
g.greet()
```
Python secretly translates:
```python
g.greet()
```
to:
```python
Greeter.greet(g)
```

In [4]:
class Greeter:
    def __init__(self, name):
        self.name = name

    def greet(self):
        print(f"Hi, I'm {self.name}")

g1 = Greeter("Alice")
g2 = Greeter("Bob")

g1.greet()
g2.greet()


Hi, I'm Alice
Hi, I'm Bob


## 5. Class Variables vs Instance Variables

**Instance variable**: stored on the instance (`self.something`).

**Class variable**: stored on the class itself.

```python
class Example:
    class_var = 0   # class variable

    def __init__(self):
        self.instance_var = 0  # instance variable
```
- `Example.class_var` is shared by all instances (unless shadowed).

- `self.instance_var` is unique per instance.

Attribute lookup order (for `obj.x`):

> instance → class → parent classes

In [5]:
class Example:
    class_var = 0

    def __init__(self):
        self.instance_var = 0

e1 = Example()
e2 = Example()

print("Initial:")
print("e1.class_var:", e1.class_var, "| e1.instance_var:", e1.instance_var)
print("e2.class_var:", e2.class_var, "| e2.instance_var:", e2.instance_var)

# Change class variable via class
Example.class_var = 10

# Change instance_var only on e1
e1.instance_var = 99

print("\nAfter changes:")
print("e1.class_var:", e1.class_var, "| e1.instance_var:", e1.instance_var)
print("e2.class_var:", e2.class_var, "| e2.instance_var:", e2.instance_var)
print("Example.class_var:", Example.class_var)

Initial:
e1.class_var: 0 | e1.instance_var: 0
e2.class_var: 0 | e2.instance_var: 0

After changes:
e1.class_var: 10 | e1.instance_var: 99
e2.class_var: 10 | e2.instance_var: 0
Example.class_var: 10


## 6. ML-Flavoured Example: A Tiny Neuron Class

We can use a class to represent something like a single neuron:

- `weights` and `bias` as instance attributes
- a `forward` method to compute the output


In [6]:
class Neuron:
    def __init__(self, weights, bias):
        self.weights = weights  # list or tuple of numbers
        self.bias = bias

    def forward(self, inputs):
        # weighted sum: sum(inputs_i * weights_i) + bias
        total = 0.0
        for x, w in zip(inputs, self.weights):
            total += x * w
        return total + self.bias

n = Neuron(weights=[0.2, 0.8, -0.5], bias=2.0)

output = n.forward([1.0, 2.0, 3.0])
print("Neuron output:", output)


Neuron output: 2.3


## 7. Representation with `__repr__`

`__repr__` defines how an object is shown when you `print()` it or inspect it in a REPL.

```python
class Neuron:
    def __init__(self, weights, bias):
        self.weights = weights
        self.bias = bias

    def __repr__(self):
        return f"Neuron(weights={self.weights}, bias={self.bias})"
```

In [7]:
class Neuron:
    def __init__(self, weights, bias):
        self.weights = weights
        self.bias = bias

    def __repr__(self):
        return f"Neuron(weights={self.weights}, bias={self.bias})"

n1 = Neuron([0.1, 0.2], 0.0)
n2 = Neuron([0.5, -0.3], 1.0)

print(n1)
print(n2)

Neuron(weights=[0.1, 0.2], bias=0.0)
Neuron(weights=[0.5, -0.3], bias=1.0)


## 8. A Mini Layer Class

We can compose multiple neurons into a simple "layer".

This isn't meant to be efficient — just to show how objects can contain other objects.


In [11]:
class Neuron:
    def __init__(self, weights, bias):
        self.weights = weights
        self.bias = bias

    def forward(self, inputs):
        # simple dot product: sum(w * x) + bias
        total = 0.0
        for w, x in zip(self.weights, inputs):
            total += w * x
        return total + self.bias

    def __repr__(self):
        return f"Neuron(weights={self.weights}, bias={self.bias})"
    
class LayerDense:
    def __init__(self, neurons):
        # neurons is a list of Neuron instances
        self.neurons = neurons

    def forward(self, inputs):
        # return list of outputs, one per neuron
        outputs = []
        for neuron in self.neurons:
            outputs.append(neuron.forward(inputs))
        return outputs

n1 = Neuron([0.2, 0.8, -0.5], 2.0)
n2 = Neuron([-0.5, 1.0, 0.1], -1.0)

layer = LayerDense([n1, n2])

print("Layer output:", layer.forward([1.0, 2.0, 3.0]))


Layer output: [2.3, 0.8]


# 08 — Exercises (Classes & Instances)

### Exercise 1 — Simple `Point` Class

Define a class:

```python
class Point:
    ...
```

with:

- attributes: `x`, `y`

- a method `distance_to_origin()` that returns the distance from (0, 0)

Use the formula:
$\text{distance} = \sqrt{x^2 + y^2}$
<br>
(you can use `**0.5` instead of `math.sqrt` for now)


In [12]:
# Excercise 1
class Point:
    ...

# Exercise 2 — `Counter` With Reset

Modify / reimplement the `Counter` class:
```python
class Counter:
    ...

```
Requirements:

- starts at 0

- has `increment()` method

- has `reset()` method that sets value back to 0

Create two counters, increment them differently, and make sure they keep separate values.

In [13]:
class Counter:
    ...

# Exercise 3 — Track Number of Instances (Class Variable)

Create a class:

```python
class Tracked:
    ...
```
that uses a class variable `count` to track how many instances have been created.

Example behaviour:
```python
t1 = Tracked()
t2 = Tracked()
print(Tracked.count)  # 2
```

In [14]:
# Excercise 3
class Tracked:
    ...


# Exercise 4 — `Dataset` Skeleton

Create a simple `Dataset` class with:

- an `__init__` that takes a list of samples and stores it

- a `__len__` method that returns how many samples there are

- a `get_sample(index)` method that returns the sample at that index

Test with a small list like `[("cat", 0), ("dog", 1)]`.

In [16]:
# Excercise 4

# Exercise 5 — `__repr__` for Debugging

Pick one of your classes (e.g. `Point`, `Counter`, or `Dataset`) and add a `__repr__` method that returns a helpful string, e.g.:
```python
Point(x=3, y=4)
```
or
```python
Dataset(num_samples=2)
```
Then create a few instances and `print()` them to see the representation.

In [18]:
# Excercise 5